In [ ]:
%reload_ext autoreload
%autoreload 2

import os

from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

## Convert from NIfTI format to Numpy Arrays

In [ ]:
"""
    Convert all .nii.gz files to .npy arrays
"""

from monai.transforms import CropForegroundd, EnsureChannelFirstD, LoadImaged, Orientationd, Compose, RandCropByPosNegLabeld

NIB_EXT = ".nii.gz"
DATA_DIR = "data/ATLAS_2/Training"
OUTPUT_SCAN_DIR = "data/ATLAS_2/Processed/Scans"
OUTPUT_MASK_DIR = "data/ATLAS_2/Processed/Masks"

os.makedirs(OUTPUT_SCAN_DIR, exist_ok=True)
os.makedirs(OUTPUT_MASK_DIR, exist_ok=True)

# get all MRI scans and masks
scans = sorted(glob(os.path.join(DATA_DIR, "**/*_T1w" + NIB_EXT), recursive=True))
masks = sorted(glob(os.path.join(DATA_DIR, "**/*T1lesion_mask" + NIB_EXT), recursive=True))

assert len(masks) == len(scans), "Length of masks and scans does not match"

# add channel dimension as the first axis, keep MRI orientation, crop the brain bounding box to exclude non-informative pixels
# images will be reshaped to (1, 192, 192, 189) - (C, H, W, D)
transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstD(keys=["image", "label"]),
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    CropForegroundd(keys=["image", "label"], source_key="image"),
    RandCropByPosNegLabeld(keys=["image", "label"], num_samples=1, spatial_size=(192, 192, -1), pos=1, neg=1, label_key="label"),
])

for scan_filepath, mask_filepath in tqdm(zip(scans, masks), total=len(scans), desc=f"Converting from {NIB_EXT} to .npy"):
    results = transforms({"image": scan_filepath, "label": mask_filepath})

    scan, mask = results[0]["image"], results[0]["label"]

    output_scan_filepath = scan_filepath.split("/")[-1].split(".")[0] + ".npy"
    output_mask_filepath = mask_filepath.split("/")[-1].split(".")[0] + ".npy"

    np.save(os.path.join(OUTPUT_SCAN_DIR, output_scan_filepath), scan)
    np.save(os.path.join(OUTPUT_MASK_DIR, output_mask_filepath), mask)


## Augmentation: CarveMix + Ecological

In [ ]:
"""
    1. Find lesion and non-lesion slices that have closely head masks
    2. Refine the previous step to slices that have the most similar shape/size to lesion slices through IoU
    3. Calculate Structural Similarity Index (SSIM) to refine the previous step. Images with an high SSIM should have a similar vasculature structure
    4. Select the lesion and apply some morphological modification (e.g., contract, expand, rotate)
    5. Standardize intensities to smooth lesion integration
"""